<a href="https://colab.research.google.com/github/dabin3178/sshs_230614/blob/main/(230614_%EC%84%9C%EC%9A%B8%EA%B3%BC%EA%B3%A0)KoBERT_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!python --version

Python 3.10.12


In [ ]:
# 설치 후 런타임 다시 시작 필요할 수도 (런타임 다시 시작 필요하면 실행창에 뜸)
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Hugging Face API로 KoBERT 다운로드
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-xgstx_o9/kobert-tokenizer_99f882f83f664b94b382c3ec433061bd
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-xgstx_o9/kobert-tokenizer_99f882f83f664b94b382c3ec433061bd
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


In [ ]:
!pip3 install kobert-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# ★ Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
# ●
#from transformers import BertTokenizer

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
# GPU 사용 확인
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
# GPU 사용 시
device = torch.device("cuda:0")

In [ ]:
# ★
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')
# ●
#bert_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
# 파라미터 세팅
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
data_pd = pd.read_csv('/content/drive/MyDrive/LAMDA/mutil_labeled_wellness_data(kkm).csv')
data_pd

,Unnamed: 0,상담 데이터,1,2,3,4,5,6,7,8,9
0,0,기분이 한없이 우울해지고 잠도 사라지고 의욕도 없어,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,너무너무 슬프고 너무너무 힘들어,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,임신하고 나서 우울하고 의욕이 없어,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,하루 종일 어디나 가지도 못하니까 눈물이 나는 거야,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,하루하루가 지옥 같아,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
9636,9636,힘이 없어서 사소한 일도 못 하겠어,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9637,9637,힘이 없어서 아무것도 안 하게 돼,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9638,9638,힘이 없어서 애한테 화내게 돼,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9639,9639,힘이 없어서 우울해,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
data_pd = data_pd.drop(data_pd.columns[[0]],axis=1)
data_pd

,상담 데이터,1,2,3,4,5,6,7,8,9
0,기분이 한없이 우울해지고 잠도 사라지고 의욕도 없어,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,너무너무 슬프고 너무너무 힘들어,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,임신하고 나서 우울하고 의욕이 없어,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,하루 종일 어디나 가지도 못하니까 눈물이 나는 거야,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,하루하루가 지옥 같아,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
9636,힘이 없어서 사소한 일도 못 하겠어,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9637,힘이 없어서 아무것도 안 하게 돼,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9638,힘이 없어서 애한테 화내게 돼,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9639,힘이 없어서 우울해,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
# 굳이 안 필요한 과정일 수도?
data_pd = data_pd.astype({'1':int, '2':int, '3':int, '4':int, '5':int, '6':int, '7':int, '8':int, '9':int})
data_pd

,상담 데이터,1,2,3,4,5,6,7,8,9
0,기분이 한없이 우울해지고 잠도 사라지고 의욕도 없어,0,1,0,0,0,0,0,0,0
1,너무너무 슬프고 너무너무 힘들어,2,1,0,0,0,0,0,0,0
2,임신하고 나서 우울하고 의욕이 없어,0,1,0,0,0,0,0,0,0
3,하루 종일 어디나 가지도 못하니까 눈물이 나는 거야,1,0,0,0,0,0,0,0,0
4,하루하루가 지옥 같아,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
9636,힘이 없어서 사소한 일도 못 하겠어,0,0,0,0,0,1,0,0,0
9637,힘이 없어서 아무것도 안 하게 돼,0,0,0,0,0,1,0,0,0
9638,힘이 없어서 애한테 화내게 돼,0,0,0,0,0,1,0,0,0
9639,힘이 없어서 우울해,0,0,0,0,0,1,0,0,0


In [ ]:
data_pd_list = data_pd.to_numpy()
data_pd_list

array([[' 기분이 한없이 우울해지고 잠도 사라지고 의욕도 없어', 0, 1, ..., 0, 0, 0],
       [' 너무너무 슬프고 너무너무 힘들어', 2, 1, ..., 0, 0, 0],
       [' 임신하고 나서 우울하고 의욕이 없어', 0, 1, ..., 0, 0, 0],
       ...,
       ['힘이 없어서 애한테 화내게 돼', 0, 0, ..., 0, 0, 0],
       ['힘이 없어서 우울해', 0, 0, ..., 0, 0, 0],
       ['힘이 없어서 죽겠어', 0, 0, ..., 0, 0, 0]], dtype=object)

In [ ]:
# 훈련, 테스트 데이터셋 분리
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_pd_list, test_size=0.25, random_state=0)

In [ ]:
dataset_train

array([['불안하고 초조한 게 계속 돼서 미치겠어', 0, 0, ..., 0, 0, 0],
       ['근데 동생의 모습을 옆에서 지켜보니까 저도 힘들더라구요.', 1, 1, ..., 0, 0, 0],
       ['아무 이유 없이 혼란스러워', 0, 0, ..., 0, 0, 0],
       ...,
       ['속상해서 소리 내고 울었어요', 1, 0, ..., 0, 0, 0],
       ['막 너무 힘들다가도 친구 힘들었던 거에 비해서는 아무 것도 아닐 거란 생각이 들어.', 1, 1, ..., 0,
        0, 0],
       ['누굴 만나면 어떻게 얘기해야 하나 걱정도 되고…', 0, 0, ..., 0, 0, 0]], dtype=object)

In [ ]:
class BERTSentenceTransform:
    r"""BERT style data transformation.

    Parameters
    ----------
    tokenizer : BERTTokenizer.
        Tokenizer for the sentences.
    max_seq_length : int.
        Maximum sequence length of the sentences.
    pad : bool, default True
        Whether to pad the sentences to maximum length.
    pair : bool, default True
        Whether to transform sentences or sentence pairs.
    """

    def __init__(self, tokenizer, max_seq_length,vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab

    def __call__(self, line):
        """Perform transformation for sequence pairs or single sequences.

        The transformation is processed in the following steps:
        - tokenize the input sequences
        - insert [CLS], [SEP] as necessary
        - generate type ids to indicate whether a token belongs to the first
        sequence or the second sequence.
        - generate valid length

        For sequence pairs, the input is a tuple of 2 strings:
        text_a, text_b.

        Inputs:
            text_a: 'is this jacksonville ?'
            text_b: 'no it is not'
        Tokenization:
            text_a: 'is this jack ##son ##ville ?'
            text_b: 'no it is not .'
        Processed:
            tokens: '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'
            type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
            valid_length: 14

        For single sequences, the input is a tuple of single string:
        text_a.

        Inputs:
            text_a: 'the dog is hairy .'
        Tokenization:
            text_a: 'the dog is hairy .'
        Processed:
            text_a: '[CLS] the dog is hairy . [SEP]'
            type_ids: 0     0   0   0  0     0 0
            valid_length: 7

        Parameters
        ----------
        line: tuple of str
            Input strings. For sequence pairs, the input is a tuple of 2 strings:
            (text_a, text_b). For single sequences, the input is a tuple of single
            string: (text_a,).

        Returns
        -------
        np.array: input token ids in 'int32', shape (batch_size, seq_length)
        np.array: valid length in 'int32', shape (batch_size,)
        np.array: input token type ids in 'int32', shape (batch_size, seq_length)

        """

        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a)
        #tokens_a = self._tokenizer(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        # The embedding vectors for `type=0` and `type=1` were learned during
        # pre-training and are added to the wordpiece embedding vector
        # (and position vector). This is not *strictly* necessary since
        # the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.

        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        #vocab = self._tokenizer.vocab
        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        # The valid length of sentences. Only real  tokens are attended to.
        valid_length = len(input_ids)

        if self._pad:
            # Zero-pad up to the sequence length.
            padding_length = self._max_seq_length - valid_length
            # use padding tokens for the rest
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')
#[출처] No module named 'kobert' 에러 해결|작성자 yeon



In [ ]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        #transform = nlp.data.BERTSentenceTransform(
        #    tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

#저는 csv 파일을 사용하여 이부분은 dataset_train.values 로 해주었습니다.
data_train = BERTDataset(dataset_train, 0, 1, tokenizer, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tokenizer, vocab, max_len, True, False)
#[출처] No module named 'kobert' 에러 해결|작성자 yeon

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
# 데이터로더 생성
batch_size = 64
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
data_train[0]

(array([   2, 2497, 7788, 4501, 7253, 7828,  921,  984, 1761, 6553, 2149,
        7483, 5405, 6855,    3,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(15, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 0)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=9,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
# 테스트용 데이터셋
test_df = pd.DataFrame({'articles':['우울해서 죽고싶어', '잠이 안 와', '요즘 자꾸 살쪄', '불안하고 심장이 두근거려', '약 먹은 것처럼 멍해', '그만 살고 싶어', '나 이제 너무 지쳤어']})
test_df

,articles
0,우울해서 죽고싶어
1,잠이 안 와
2,요즘 자꾸 살쪄
3,불안하고 심장이 두근거려
4,약 먹은 것처럼 멍해
5,그만 살고 싶어
6,나 이제 너무 지쳤어


In [ ]:
# 테스트용 데이터셋
test_df = pd.DataFrame({'상담 데이터':['우울해서 죽고싶어', '잠이 안 와', '요즘 자꾸 살쪄', '불안하고 심장이 두근거려', '약 먹은 것처럼 멍해', '그만 살고 싶어', '나 이제 너무 지쳤어']}, columns=data_pd.columns)
test_df

,상담 데이터,1,2,3,4,5,6,7,8,9
0,우울해서 죽고싶어,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,잠이 안 와,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,요즘 자꾸 살쪄,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,불안하고 심장이 두근거려,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,약 먹은 것처럼 멍해,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,그만 살고 싶어,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,나 이제 너무 지쳤어,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# KoBERT 모델 이용하여 멀티 라벨 분류

In [ ]:
# KoBERT 모델로 멀티 라벨 결과 예측하기

import numpy as np
import math


def predict(df):

    for i in range(0, len(df)): # 게시글 총 문장 개수만큼 반복
      if i % 100 == 0:
        print(i)

      data = [df.iloc[i, 0], '0']
      #data = [df.iloc[i, 0]]
      print(data)
      dataset_another = [data]
      another_test = BERTDataset(dataset_another, 0,1, tokenizer,vocab, max_len, True, False)
      test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)


      model.eval()

      for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
          token_ids = token_ids.long().to(device)
          segment_ids = segment_ids.long().to(device)

          valid_length= valid_length
          label = label.long().to(device)

          out = model(token_ids, valid_length, segment_ids)

          for j in out:
            #logits = j
            probability = torch.sigmoid(j).cpu().detach().numpy() # 시그모이드 함수 사용해 0과 1 사이값으로 변환
            #print(torch.sigmoid(j))

            # 시그모이드 변환값이 0.5이상이라면 라벨 할당
            temp = [] # 라벨 저장할 리스트
            for p in probability:
              if p > 0.5:
                temp.append(1)
              else:
                temp.append(0)
            print(temp)

            df.iloc[i, 1:] = temp
            #df.iloc[i, 1:] = temp[:len(df.columns)-1]
            #nn =(df.shape[1]-1)/ 2
            #df.iloc[i, 1:] = np.tile(temp, (1, nn+1))
            #df.iloc[i, 1:len(temp)+1] = temp

            #logits = logits.detach().cpu().numpy() # 라벨 예측 결과

            #df.iloc[i, 1:] = np.argmax(logits)
            #print(df.iloc[i, 1])

In [ ]:
predict(test_df)

0
['우울해서 죽고싶어', '0']


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


[1, 1, 0, 0, 0, 1, 0, 0, 1]
['잠이 안 와', '0']
[1, 1, 0, 0, 0, 0, 0, 0, 1]
['요즘 자꾸 살쪄', '0']
[1, 1, 0, 1, 0, 1, 0, 0, 1]
['불안하고 심장이 두근거려', '0']
[1, 1, 0, 0, 0, 1, 1, 1, 1]
['약 먹은 것처럼 멍해', '0']
[0, 0, 0, 1, 1, 1, 1, 0, 0]
['그만 살고 싶어', '0']
[1, 1, 0, 0, 0, 0, 0, 0, 1]
['나 이제 너무 지쳤어', '0']
[1, 1, 0, 0, 0, 1, 0, 0, 1]


# 각 라벨별 확률 구하기 (Sigmoid)

In [ ]:
# KoBERT 모델로 예측한 에브리타임 게시물 분류 결과 저장_확률편

import numpy as np

def predict_prb(df):

    for i in range(0, len(df)): # 게시글 총 문장 개수만큼 반복
      if i % 100 == 0:
        print(i)
      data = [df.iloc[i, 0], '0']
      #data = [df.iloc[i, 0]]
      print(data)
      dataset_another = [data]
      another_test = BERTDataset(dataset_another, 0,1, tokenizer,vocab, max_len, True, False)
      test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

      model.eval()

      for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
          token_ids = token_ids.long().to(device)
          segment_ids = segment_ids.long().to(device)

          valid_length= valid_length
          label = label.long().to(device)

          out = model(token_ids, valid_length, segment_ids)

          for j in out:
            logits = j
            probability = torch.sigmoid(j).cpu().detach().numpy() # 시그모이드 함수 사용해 0과 1 사이값으로 변환
            #print(torch.sigmoid(j))
            print(probability)

'''
            # 시그모이드 변환값이 0.5이상이라면 라벨 할당
            temp = [] # 라벨 저장할 리스트
            for p in probability:
              if p > 0.5:
                temp.append(1)
              else:
                temp.append(0)
            print(temp)
'''

            #df.iloc[i, 1:] = temp
            #df.iloc[i, 1:] = temp[:len(df.columns)-1]

            # logits = logits.detach().cpu().numpy() # 라벨 예측 결과

            # df.iloc[i, 1] = np.argmax(logits)

'\n            # 시그모이드 변환값이 0.5이상이라면 라벨 할당\n            temp = [] # 라벨 저장할 리스트\n            for p in probability:\n              if p > 0.5:\n                temp.append(1)\n              else:\n                temp.append(0)\n            print(temp)\n'

In [ ]:
# 테스트 진행
predict_prb(test_df)

0
['우울해서 죽고싶어', '0']
[0.5451897  0.641793   0.44958043 0.47445557 0.45030192 0.5418367
 0.44043186 0.39248124 0.5363399 ]
['잠이 안 와', '0']
[0.51914704 0.5365337  0.46549547 0.43592715 0.42859027 0.48921758
 0.47205347 0.49209717 0.53383005]
['요즘 자꾸 살쪄', '0']
[0.53282344 0.63840324 0.42938158 0.5237184  0.4986961  0.579208
 0.4689829  0.34996903 0.54427975]
['불안하고 심장이 두근거려', '0']
[0.52071524 0.5135185  0.47952756 0.4471524  0.44707578 0.5243407
 0.5651563  0.5004648  0.5498563 ]
['약 먹은 것처럼 멍해', '0']
[0.4859557  0.49956456 0.4291864  0.54904145 0.5762055  0.5268602
 0.54319483 0.40171608 0.41170588]
['그만 살고 싶어', '0']
[0.5541959  0.6151639  0.46520582 0.44913042 0.43083102 0.4950511
 0.44034743 0.4253814  0.55477136]
['나 이제 너무 지쳤어', '0']
[0.5427215  0.63659006 0.44400388 0.4836218  0.467699   0.5428569
 0.44121513 0.39075828 0.52416825]


In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))


<ipython-input-30-35123b067f85>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/113 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.3523240089416504 train acc 0.03125
epoch 1 train acc 0.6536049457607765


<ipython-input-30-35123b067f85>:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/38 [00:00<?, ?it/s]

...

epoch 4 test acc 0.9576575887392902


  0%|          | 0/113 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.015720907598733902 train acc 1.0
epoch 5 train acc 0.9901825221238938


  0%|          | 0/38 [00:00<?, ?it/s]

epoch 5 test acc 0.9677268206854345
